# MlFlow

In [1]:
import mlflow
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

C:\Users\Utilisateur\.conda\envs\ABD\lib\site-packages\numpy\__init__.py:148: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init


ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.9 from "C:\Users\Utilisateur\.conda\envs\ABD\python.exe"
  * The NumPy version is: "1.21.2"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: Le module spécifié est introuvable.


### Retrieving the csv data in the notebook 

In [ ]:
df_trimed = pd.read_csv("../Data/dataset_final.csv")
x_train, x_test, y_train, y_test = train_test_split(df_trimed.loc[:,df_trimed.columns != 'TARGET'], df_trimed['TARGET'], test_size = 0.3, random_state = 50)

### Creation of a function evaluate, to create logs in the MlFlow ui

In [ ]:
def evaluate(x_test, y_test, model):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    import mlflow
    import mlflow.sklearn
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    with mlflow.start_run():

        # Evaluate Metrics
        predicted_qualities = model.predict(x_test)
        (rmse, mae, r2) = eval_metrics(y_test, predicted_qualities)

        # Print out metrics
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(model, "model")

### Definition of the URL of MlFlow logs

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000") ## Use the web-link in the uri to open the mlflow ui

### Use of the evaluate fonction

In [ ]:
XGBoost = pickle.load(open('../Model/XGBoost.pkl', 'rb'))
run = evaluate(x_test, y_test, XGBoost)
run